In [1]:
# Importing the required library for handling HDF5 files
import h5py

# Opening the HDF5 file
file_path = '/home/ubuntu/data/de_111496386_20211021101732_20211021102136.hd5'
hdf5_file = h5py.File(file_path, 'r')

# Function to explore the structure of the HDF5 file recursively
def explore_hdf5_group(group, indent=0):
    items = []
    for key, value in group.items():
        if isinstance(value, h5py.Group):
            items.append(("  " * indent + f"Group: {key}", explore_hdf5_group(value, indent + 1)))
        elif isinstance(value, h5py.Dataset):
            items.append({key})
            #print({key})
        else:
            items.append(("  " * indent + f"Unknown type: {key}", str(type(value))))
            print({key})
    return items

# Exploring the root of the HDF5 file
structure = explore_hdf5_group(hdf5_file)
print("structure:",structure)

# Function to extract metadata from the HDF5 group
def extract_metadata(group):
    metadata = {}
    for key, value in group.items():
        # Exclude the datasets that represent ECG leads
        #if key not in ["I", "II", "III", "V1", "V2", "V3", "V4", "V5", "V6", "aVF", "aVL", "aVR"]:
        if value.shape == ():  # Check if the dataset is a scalar value
            metadata[key] = value[()]
            print(key,metadata[key])
        else:
            metadata[key] = value[:]  # For non-scalar values, get the entire array
            #print(key,metadata[key])
    return metadata
'''
# Extracting metadata from the specific group
t = hdf5_file['ecg'].keys()
print(t)
ecg_group = hdf5_file['ecg'][t]
metadata = extract_metadata(ecg_group)
print("metadata:",metadata)
'''


for k in list(hdf5_file['ecg'].keys()):
    t = hdf5_file['ecg'][k]
    ecg = []
    metadata = extract_metadata(t)




structure: [('Group: ecg', [('  Group: 2021-10-21T00:00:00', [{'I'}, {'II'}, {'III'}, {'V1'}, {'V2'}, {'V3'}, {'V4'}, {'V5'}, {'V6'}, {'aVF'}, {'aVL'}, {'aVR'}, {'admitdate'}, {'admittime'}, {'atrialrate_md'}, {'atrialrate_pc'}, {'cartnumber'}, {'datatype'}, {'dateofbirth'}, {'ecgsamplebase_md'}, {'ecgsamplebase_pc'}, {'ecgsampleexponent_md'}, {'ecgsampleexponent_pc'}, {'editdate'}, {'editliststatus'}, {'edittime'}, {'gender'}, {'hisdisposition'}, {'hisstatus'}, {'orderplacementdate'}, {'orderplacementtime'}, {'ordertime'}, {'patientid'}, {'patientid_clean'}, {'paxis_md'}, {'paxis_pc'}, {'poffset_md'}, {'poffset_pc'}, {'ponset_md'}, {'ponset_pc'}, {'printerval_md'}, {'printerval_pc'}, {'priority'}, {'qoffset_md'}, {'qoffset_pc'}, {'qonset_md'}, {'qonset_pc'}, {'qrscount_md'}, {'qrscount_pc'}, {'qrsduration_md'}, {'qrsduration_pc'}, {'qtcfrederica_md'}, {'qtcfrederica_pc'}, {'qtcorrected_md'}, {'qtcorrected_pc'}, {'qtinterval_md'}, {'qtinterval_pc'}, {'raxis_md'}, {'raxis_pc'}, {'referr

In [3]:
import os
import pandas as pd

def process_and_combine_excel_files(directory_path):
    # List all files in the directory
    all_files = os.listdir(directory_path)
    excel_files = [f for f in all_files if f.endswith('.xlsx') or f.endswith('.xls')]

    # Read and process each excel file
    dataframes = []
    for file in excel_files:
        file_path = os.path.join(directory_path, file)
        df = pd.read_excel(file_path)
        # Replace "|" with a space
        df['deid_t_diagnosis_original'] = df['deid_t_diagnosis_original'].str.replace('|', ' ')
        dataframes.append(df)

    # Combine all dataframes
    combined_df = pd.concat(dataframes, ignore_index=True)
    
    # Save combined dataframe to a CSV
    combined_csv_path = os.path.join(directory_path, 'combined_ecg_diagnosis.csv')
    combined_df.to_csv(combined_csv_path, index=False)
    
    return combined_csv_path

combined_file_path = "/home/ubuntu/data/ECG/"
process_and_combine_excel_files(combined_file_path)

'/home/ubuntu/data/ECG/combined_ecg_diagnosis.csv'

In [7]:

import pandas as pd

def filter_csv_based_on_files(csv_path, folder_path):
    # 读取CSV文件
    df = pd.read_csv(csv_path)
    
    # 去除NaN值
    df = df.dropna(subset=['HashFileName'])
    
    # 检查每个'HashFileName'是否在文件夹中存在
    df = df[df['HashFileName'].apply(lambda x: os.path.exists(os.path.join(folder_path, x)))]
    
    # 保存经过处理的CSV文件
    output_path = os.path.join(os.path.dirname(csv_path), "filtered_" + os.path.basename(csv_path))
    df.to_csv(output_path, index=False)
    
    return output_path

# 使用方法：
output_file = filter_csv_based_on_files('/home/ubuntu/data/ECG/combined_ecg_diagnosis.csv', '/home/ubuntu/data/ECG/2019/02/')
